Team Submission – ARISE Data Cleaning (Missing + Inconsistencies)
Team members:Teja Sree, Anudeepthi,Rupali

What this notebook does (Week submission):
1) Load ARISE survey data
2) Standardize missing values (blank cells, NA/N/A text, and coded missing numbers like 99/999)
3) Check for inconsistent values (ex: Q8/Q119/Q5 should only be 1–5)
4) Fix inconsistencies by converting invalid values to missing (NA)
5) Create core harmonized columns (workforce, fiscal, grant capacity)
6) Save cleaned dataset + missingness report

In [28]:
import pandas as pd 
import numpy as np

In [29]:
file_name = "ARISE_Dataset.csv"
arise = pd.read_csv(file_name)


In [30]:
print("Loaded:", arise.shape)
arise.head()

Loaded: (309, 91)


,Unnamed: 0,ResponseId,RecipientLastName,ExternalReference,1st Dis,2nd Dis,3rd Dis,Q3_10_Agri,Q3_9_Cyber,Q3_19_Dam,...,Q13_Reduce staff,Q13_Defer capital projects,Q13_Reduce benefits,Q13_Reduce services,Q13_Increase taxes,Q13_Increase user fees,Q13_Adopt new fees,Q13_Reduce fund balance,Q13_Defer maintenance expenditures,Q13_None of the above
0,1,R_2sQNn7iItA9jdBn,chief administrator,2079950,3.0,21.0,1.0,NaN,NaN,NaN,...,1,1,1,0,1,1,1,0,0,0
1,2,R_2Qyp1YFLCR17M9C,chief administrator,2061250,11.0,3.0,8.0,NaN,NaN,NaN,...,1,1,0,1,1,0,0,0,1,0
2,3,R_2aQAihYklmvMFey,chief administrator,2005600,11.0,21.0,1.0,NaN,NaN,NaN,...,1,1,0,1,1,0,0,0,1,0
3,4,R_6sTVkGKJ3zy6aPL,chief administrator,2034300,21.0,1.0,2.0,NaN,NaN,NaN,...,0,1,0,0,1,0,0,0,0,0
4,5,R_3sddVtnZ4CSdNk0,chief administrator,2053225,3.0,1.0,23.0,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,0


In [31]:
if "Unnamed: 0" in arise.columns:
    arise.drop(columns=["Unnamed: 0"], inplace=True)
    print("Removed extra column: Unnamed: 0")

Removed extra column: Unnamed: 0


In [38]:
# 1) Blank spaces → NaN
arise = arise.replace(r"^\s*$", np.nan, regex=True)

# 2) Common missing text → NaN
arise = arise.replace(
    ["NA","N/A","na","n/a","NULL","null","None","none","Don't know","Dont know","DK","Refused","Prefer not to say"],
    np.nan
)

# 3) Common numeric missing codes → NaN
arise = arise.replace({99: np.nan, 999: np.nan, 9999: np.nan, -99: np.nan, -999: np.nan})

print("Missing values standardized (blank/text/codes).")

Missing values standardized (blank/text/codes).


In [40]:
missing_percent = (arise.isna().mean() * 100).sort_values(ascending=False)
missing_count = arise.isna().sum().sort_values(ascending=False)

missing_report = pd.DataFrame({
    "missing_count": missing_count,
    "missing_percent": missing_percent.round(2)
})

print("Top 15 columns with highest missing %:")
display(missing_report.head(15))

Top 15 columns with highest missing %:


,missing_count,missing_percent
Q3_17_Terrorism,309,100.00
Q3_12_Earthquake,309,100.00
Q3_19_Dam,306,99.03
Q3_7_waste spill,305,98.71
Q3_8_Industrial fire,305,98.71
Q3_10_Agri,302,97.73
Q3_23_Others,301,97.41
Q3_23_TEXT,296,95.79
Q3_22_Soil erosion,291,94.17
Q3_5_Tornados,289,93.53


In [46]:
# These are the key ARISE variables from our crosswalk.
for col in ["Q8", "Q119", "Q5"]:
    print(col, "found" if col in arise.columns else "NOT found")

Q8 found
Q119 found
Q5 found


In [48]:
# We check if any values fall outside 1–5 (ex: 0, 6, 23, etc.)

for col in ["Q8", "Q119", "Q5"]:
    if col in arise.columns:
        x = pd.to_numeric(arise[col], errors="coerce")
        bad = x.notna() & ((x < 1) | (x > 5))
        print(f"\n{col}: invalid values outside 1–5 =", bad.sum())
        if bad.any():
            print("Invalid sample values:", sorted(x[bad].unique())[:10])


Q8: invalid values outside 1–5 = 0

Q119: invalid values outside 1–5 = 0

Q5: invalid values outside 1–5 = 35
Invalid sample values: [6.0]


In [50]:
# If a value is not between 1–5, we treat it as invalid and convert it to missing (NaN).

for col in ["Q8", "Q119", "Q5"]:
    if col in arise.columns:
        x = pd.to_numeric(arise[col], errors="coerce")
        arise[col] = x.where((x >= 1) & (x <= 5), np.nan)

print("Fixed: Q8, Q119, Q5 now contain only valid 1–5 values (or NaN).")

Fixed: Q8, Q119, Q5 now contain only valid 1–5 values (or NaN).


In [58]:
# If a value is not between 1–5, we treat it as invalid and convert it to missing (NaN).

for col in ["Q8", "Q119", "Q5"]:
    if col in arise.columns:
        x = pd.to_numeric(arise[col], errors="coerce")
        arise[col] = x.where((x >= 1) & (x <= 5), np.nan)

print("Fixed: Q8, Q119, Q5 now contain only valid 1–5 values (or NaN).")

Fixed: Q8, Q119, Q5 now contain only valid 1–5 values (or NaN).


In [60]:
# These are the cleaned harmonized variables we will use in analysis.

arise["workforce_challenge_1to5"] = arise["Q8"] if "Q8" in arise.columns else np.nan
arise["fiscal_condition_1to5"] = arise["Q119"] if "Q119" in arise.columns else np.nan
arise["grant_capacity_1to5"] = arise["Q5"] if "Q5" in arise.columns else np.nan

arise["survey_source"] = "ARISE"

arise[["workforce_challenge_1to5","fiscal_condition_1to5","grant_capacity_1to5","survey_source"]].head()

,workforce_challenge_1to5,fiscal_condition_1to5,grant_capacity_1to5,survey_source
0,3.0,3.0,3.0,ARISE
1,4.0,4.0,2.0,ARISE
2,5.0,4.0,3.0,ARISE
3,3.0,3.0,2.0,ARISE
4,3.0,3.0,3.0,ARISE


In [66]:
# These are preparedness (Q9) and vulnerable support (Q10) items

q9_cols = [c for c in arise.columns if c.startswith("Q9_")]
q10_cols = [c for c in arise.columns if c.startswith("Q10_")]

print("Q9 columns found:", len(q9_cols))
print(q9_cols)

print("\nQ10 columns found:", len(q10_cols))
print(q10_cols)

Q9 columns found: 14
['Q9_Early warning', 'Q9_Evacuation plan', 'Q9_Financial assistance for low-income AC', 'Q9_Water conservation programs', 'Q9_Energy conservation programs', 'Q9_Zoning', 'Q9_Financial assistance for low-income shut-offs', 'Q9_Heating or cooling stations', 'Q9_Tornado shelter', 'Q9_Early warning_Lang', 'Q9_Code enforcement', 'Q9_Backup electric', 'Q9_Evacuation route or plan', 'Q9_None of the above']

Q10 columns found: 8
['Q10_Elderly people', 'Q10_Low income', 'Q10_Homeless', 'Q10_Non-English', 'Q10_Racial minorities', 'Q10_Disabled', 'Q10_Immigrants', 'Q10_None of the above']


In [68]:
# cleaning Q9 Q10

def clean_yes_no(series):
    x = pd.to_numeric(series, errors="coerce")
    # Keep only 0 and 1, everything else becomes NaN (safer for survey data)
    return x.where(x.isin([0, 1]), np.nan)

# Apply cleaning to Q9 and Q10 columns
for col in q9_cols:
    arise[col] = clean_yes_no(arise[col])

for col in q10_cols:
    arise[col] = clean_yes_no(arise[col])

print("Cleaned Q9 and Q10 values (missing preserved correctly).")

Cleaned Q9 and Q10 values (missing preserved correctly).


In [70]:
arise["preparedness_score"] = arise[q9_cols].sum(axis=1, min_count=1)
arise["vulnerable_support_score"] = arise[q10_cols].sum(axis=1, min_count=1)

# Quick preview
arise[["preparedness_score", "vulnerable_support_score"]].head()

,preparedness_score,vulnerable_support_score
0,3,1
1,5,1
2,4,1
3,9,3
4,6,1


In [72]:
arise.to_csv("ARISE_cleaned_missing_fixed.csv", index=False)